In [29]:
from pathlib import Path
import sys  

# Get my_package directory path from Notebook
parent_dir = str(Path().resolve().parents[0])
print(parent_dir)
# Add to sys.path

path_set = set(sys.path)
if parent_dir not in path_set:
    sys.path.insert(0, parent_dir)

print(sys.path)

C:\Users\clorch\Documents\education-benchmarking-and-insights\data-pipeline
['C:\\Users\\clorch\\Documents\\education-benchmarking-and-insights\\data-pipeline', 'c:\\Users\\clorch\\Documents\\education-benchmarking-and-insights\\data-pipeline\\workbooks', 'c:\\Users\\clorch\\AppData\\Local\\anaconda3\\python311.zip', 'c:\\Users\\clorch\\AppData\\Local\\anaconda3\\DLLs', 'c:\\Users\\clorch\\AppData\\Local\\anaconda3\\Lib', 'c:\\Users\\clorch\\AppData\\Local\\anaconda3', '', 'c:\\Users\\clorch\\AppData\\Local\\anaconda3\\Lib\\site-packages', 'c:\\Users\\clorch\\AppData\\Local\\anaconda3\\Lib\\site-packages\\win32', 'c:\\Users\\clorch\\AppData\\Local\\anaconda3\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\clorch\\AppData\\Local\\anaconda3\\Lib\\site-packages\\Pythonwin']


# VMFI Data processing pipeline

This workbook aims to emulate the current data processing pipeline that occurs in VMFI pipeline. The logic and processing is largely based on the following document [Insights data portal - Data sources and sql analysis](https://educationgovuk.sharepoint.com.mcas.ms/:w:/r/sites/VMFI/_layouts/15/Doc.aspx?sourcedoc=%7B38C1DC37-7CDB-48B8-9E22-284F4F311C0B%7D&file=1.%20Insights%20portal%20-%20data%20sources%20and%20sql%20analysis%20v010%20-%20Copy.docx&action=default&mobileredirect=true) and will stay true to this document even if the existing stored procedures are doing something different. This will form the basis of a gap analysis going forward. 

All data loaded in the following workbook comes from the set of CSV files in the `data` folder alongside this workbook. These datasets are for the most part from the list at the start of the linked document. However, because there is additional standing data required to fully implement the pipeline then this data has been exported from the development VMFI pipeline database. These files are currently: 

| File name | DB Table |
|:----------|----------|
|standing_data_cdc.csv | standing_data.cdc |

In [30]:
import src.pipeline.pre_processing as pre_processing
import src.pipeline.output_schemas as output_schemas
import pandas as pd
import time
import glob
import os

In [31]:
# Create and clean directory
from pathlib import Path
Path("output/pre-processing").mkdir(parents=True, exist_ok=True)

files = glob.glob("output/pre-processing/*")
for f in files:
    os.remove(f)

In [32]:
start_time = time.time()
current_year = 2022

## CDC data load and preparation

School buildings condition dataset. Based on the surveys performed throughout 2018-2019.

The data in the file `data/standing_data_cdc.csv` is just an export of the data in `standing_data.cdc` table. Without the Year and Import ID fields. In future this will likely have to be read directly from the source database as per [this document.](https://educationgovuk.sharepoint.com.mcas.ms/:w:/r/sites/VMFI/_layouts/15/Doc.aspx?sourcedoc=%7B38C1DC37-7CDB-48B8-9E22-284F4F311C0B%7D&file=1.%20Insights%20portal%20-%20data%20sources%20and%20sql%20analysis%20v010%20-%20Copy.docx&action=default&mobileredirect=true) 

In [33]:
cdc = pre_processing.prepare_cdc_data('data/cdc.csv', current_year)

In [34]:
#cdc.to_csv('output/pre-processing/cdc.csv')
cdc

,Total Internal Floor Area,Age Average Score
URN,,
100150,2803.0,48.358188
100162,2105.0,133.162945
100164,2934.0,97.0
100166,2040.0,91.705882
105304,1602.0,35.752809
...,...,...
144913,3111.0,16.704275
144917,2620.0,78.412214
105623,3382.0,7.0


## School Census data load

*Pupil Census* - DfE data collection providing information about school and pupil characteristics, for example percentage of pupils claiming free school`z meals, or having English as their second language. 

*Workforce census* - Single reference for all school workforce statistics based on staff working in publicly funded schools in England.

The following code loads both the workforce and pupil census data and preforms an `inner` join by URN on the data sets.

In [35]:
census = pre_processing.prepare_census_data('data/census_workforce.xlsx', 'data/census_pupils.csv')

#todo - add logic for highest / lowest age from census data

In [36]:
#census.to_csv('output/pre-processing/census.csv')
census

,region_name,district_administrative_name,ward_name,full time pupils,Percentage claiming Free school meals,Percentage Free school meals,number of pupils whose first language is known or believed to be other than English,Total School Workforce (Headcount),Total Number of Teachers in the Leadership Group (Headcount),Total Number of Teachers (Headcount),...,Total Number of Auxiliary Staff (Headcount),Total School Workforce (Full-Time Equivalent),Total Number of Teachers in the Leadership Group (Full-time Equivalent),Total Number of Teachers (Full-Time Equivalent),Total Number of Teaching Assistants (Full-Time Equivalent),FullTimeOther,Total Number of Auxiliary Staff (Full-Time Equivalent),Pupil: Teacher Ratio (Full-Time Equivalent of qualified and unqualified teachers),Teachers with Qualified Teacher Status (%) (Headcount),Number of Vacant Teacher Posts
URN,,,,,,,,,,,,,,,,,,,,,
141334,East Midlands,Nottingham,Bilborough,325.0,33.8,52.3,93.0,48,3,15,...,13,34.17,2.64,13.11,10.29,4.82,5.95,24.8,100.000000,0
141396,East Midlands,Nottingham,Aspley,599.0,23.4,60.3,236.0,118,4,39,...,34,82.47,4.00,34.00,29.55,10.13,8.79,18.3,100.000000,0
141397,East Midlands,Nottingham,Bilborough,465.0,33.2,47.7,127.0,105,5,27,...,27,72.81,4.24,24.55,31.84,6.55,9.87,19.7,100.000000,0
142223,East Midlands,Nottingham,Wollaton West,1050.0,5.1,8.7,343.0,156,5,56,...,47,99.66,4.16,47.12,33.07,6.57,12.90,23.0,100.000000,0
144396,East Midlands,Nottingham,Bulwell,196.0,56.7,64.8,29.0,37,2,13,...,11,25.57,2.00,11.39,7.36,4.00,2.82,18.1,100.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104642,North West,Liverpool,Church,421.0,2.4,2.6,14.0,52,4,18,...,18,34.47,3.60,15.80,6.27,5.22,7.18,26.6,100.000000,0
104643,North West,Liverpool,Cressington,426.0,3.5,8.5,13.0,68,3,19,...,19,39.89,3.00,17.40,11.74,3.34,7.41,24.7,100.000000,0
104645,North West,Liverpool,Tuebrook and Stoneycroft,237.0,32.9,33.8,43.0,37,3,13,...,10,26.47,3.00,12.40,6.78,3.19,4.10,19.1,92.307692,0


## Special Education Needs (SEN) data load and preparation

Special educational needs dataset. Contains information about the number of pupils, who require various SEN provisions. This loads the `SEN` data, which originates from [here](https://explore-education-statistics.service.gov.uk/find-statistics/special-educational-needs-in-england#dataDownloads-1)

In [37]:
sen = pre_processing.prepare_sen_data('data/sen.csv')

In [38]:
#sen.to_csv("output/pre-processing/sen.csv")
sen

,Total pupils,EHC plan,Percentage SEN,Primary Need SPLD,Primary Need MLD,Primary Need SLD,Primary Need PMLD,Primary Need SEMH,Primary Need SLCN,Primary Need HI,...,Percentage Primary Need SLD,Percentage Primary Need PMLD,Percentage Primary Need SEMH,Percentage Primary Need SLCN,Percentage Primary Need HI,Percentage Primary Need VI,Percentage Primary Need MSI,Percentage Primary Need PD,Percentage Primary Need ASD,Percentage Primary Need OTH
URN,,,,,,,,,,,,,,,,,,,,,
100000,271.0,8.0,2.952030,2.0,4.0,0.0,0.0,9.0,31.0,2.0,...,0.0,0.000000,3.321033,11.439114,0.738007,0.000000,0.738007,0.000000,3.321033,0.369004
100001,739.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
100002,269.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
100003,1045.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
100005,136.0,2.0,1.470588,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.735294,0.000000,0.000000,0.000000,0.000000,0.000000,0.735294,16.176471,0.735294
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149557,41.0,3.0,7.317073,2.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.000000,0.000000,2.439024,0.000000,0.000000,0.000000,0.000000,4.878049,0.000000
149632,1291.0,58.0,4.492641,31.0,15.0,0.0,0.0,20.0,25.0,8.0,...,0.0,0.000000,1.549187,1.936483,0.619675,0.309837,0.077459,0.542215,2.013943,1.781565
149633,86.0,0.0,0.000000,2.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0.0,0.000000,1.162791,1.162791,0.000000,0.000000,0.000000,0.000000,1.162791,1.162791


## KS2 and KS4 processing

In [39]:
#ks2 = pre_processing.prepare_ks2_data('data/ks2.xlsx')
ks2 = pd.DataFrame({'URN':[100,200]})

In [40]:
#ks2.to_csv('output/pre-processing/ks2.csv')
ks2

,URN
0,100
1,200


In [41]:
#ks4 = pre_processing.prepare_ks4_data('data/ks4.xlsx')
ks4 = pd.DataFrame({'URN':[100,200]})

In [42]:
#ks4.to_csv('output/pre-processing/ks4.csv')
ks4

,URN
0,100
1,200


## AR Data load and preparation

This loads the Annual accounts return dataset and the corresponding mapping file. This extract only contains benchmarking section, which consists of submissions of costs, income, and balances of individual academies.

The mapping file, contains the mapping from AR4 cell references to cost categories and descriptions.

In [43]:
academy_ar = pre_processing.prepare_aar_data('data/academy_ar.xlsx')

In [44]:
#academy_ar.to_csv('output/pre-processing/academy_ar.csv')
academy_ar

,Trust UPIN,London Weighting,PFI School,DFE/EFA Revenue grants (includes Coronavirus Government Funding,of which: Coronavirus Government Funding,SEN funding,Other DfE/EFA Revenue Grants,Other income - LA & other Government grants,"Government source, non-grant",Academies,...,Trust_All income from facilities and services,Trust_Income from catering,Trust_Receipts from supply teacher insurance claims,Trust_Donations and/or voluntary funds,Trust_Other self-generated income,Trust_Investment income,Central Services Balance,Central Services Financial Position,Academy Financial Position,Trust Financial Position
Academy UPIN,,,,,,,,,,,,,,,,,,,,,
111443,137157,Neither,Non-PFI school,7967000.0,41000.0,153000.0,262000.0,0.0,0.0,0.0,...,539000.0,1063000.0,0.0,127000.0,473000.0,0.0,-1830000.0,Deficit,Deficit,Deficit
111451,138199,Neither,Non-PFI school,6342000.0,80000.0,222000.0,7000.0,203000.0,0.0,0.0,...,442000.0,1000.0,0.0,702000.0,0.0,0.0,-8541000.0,Deficit,Deficit,Surplus
111453,135112,Neither,Non-PFI school,2798000.0,25000.0,162000.0,63000.0,0.0,0.0,0.0,...,286000.0,82000.0,0.0,426000.0,0.0,0.0,-3191000.0,Deficit,Surplus,Surplus
111710,135428,Neither,Non-PFI school,7685000.0,79000.0,323000.0,215000.0,83000.0,0.0,0.0,...,96000.0,252000.0,0.0,19000.0,9000.0,15000.0,0.0,Deficit,Deficit,Deficit
113087,136879,Neither,Non-PFI school,8021000.0,0.0,93000.0,45000.0,81000.0,0.0,0.0,...,77000.0,0.0,0.0,14000.0,666000.0,26000.0,0.0,Deficit,Deficit,Deficit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164746,140031,Neither,Non-PFI school,1103000.0,76000.0,1213000.0,123000.0,18000.0,0.0,0.0,...,17000.0,78000.0,0.0,1000.0,361000.0,0.0,-1067000.0,Deficit,Deficit,Surplus
164811,135065,Neither,Non-PFI school,54000.0,0.0,33000.0,0.0,1000.0,0.0,0.0,...,2168000.0,334000.0,0.0,525000.0,0.0,0.0,-8019000.0,Deficit,Deficit,Surplus
164811,140031,Neither,Non-PFI school,949000.0,11000.0,1219000.0,0.0,37000.0,0.0,0.0,...,17000.0,78000.0,0.0,1000.0,361000.0,0.0,-1067000.0,Deficit,Deficit,Surplus


Create a summary table for the AR stance of each distinct academy in the table.

Now compute the trust financial position in the same manor as the individual academy position

## Academy and maintained schools data load and preparation

This reads the main GIAS data (edubasealldataYYYYMMDD file) and the associated links file (links_edubasealldataYYYYMMDD file). This is taken from the [GIAS Service](https://get-information-schools.service.gov.uk/help)

Other columns are tidied up by asserting the correct type for that column. This is tidying phase is largly because on load integer columns will be inferred to be a float as opposed to an integer.

In [45]:
schools = pre_processing.prepare_schools_data('data/gias.csv','data/gias_links.csv')


In [46]:
#schools.to_csv('output/pre-processing/schools.csv')
schools.sort_index()

,LA (code),LA (name),EstablishmentNumber,EstablishmentName,TypeOfEstablishment (code),TypeOfEstablishment (name),EstablishmentStatus (code),EstablishmentStatus (name),OpenDate,CloseDate,...,UrbanRural (name),BoardingEstablishment (name),PreviousLA (code),PreviousLA (name),PreviousEstablishmentNumber,OfstedRating (name),MSOA (code),LSOA (code),LA Establishment Number,HeadName
URN,,,,,,,,,,,,,,,,,,,,,
100000,201,City of London,3614,The Aldgate School,2,Voluntary aided school,1,Open,NaT,NaT,...,(England/Wales) Urban major conurbation,<NA>,999,<NA>,<NA>,Outstanding,E02000001,E01032739,201-3614,Miss Alexandra Allan
100001,201,City of London,6005,City of London School for Girls,11,Other independent school,1,Open,1920-01-01,NaT,...,(England/Wales) Urban major conurbation,Does not have boarders,999,<NA>,<NA>,,E02000001,E01000002,201-6005,Mrs Jenny Brown
100002,201,City of London,6006,St Paul's Cathedral School,11,Other independent school,1,Open,1939-01-01,NaT,...,(England/Wales) Urban major conurbation,Has boarders,999,<NA>,<NA>,,E02000001,E01032739,201-6006,<NA>
100003,201,City of London,6007,City of London School,11,Other independent school,1,Open,1919-01-01,NaT,...,(England/Wales) Urban major conurbation,Does not have boarders,999,<NA>,<NA>,,E02000001,E01032739,201-6007,Mr Alan Bird
100005,202,Camden,1048,Thomas Coram Centre,15,Local authority nursery school,1,Open,NaT,NaT,...,(England/Wales) Urban major conurbation,<NA>,999,<NA>,<NA>,Outstanding,E02007115,E01000937,202-1048,Ms Perina Holness
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402468,679,Monmouthshire,5500,King Henry viii 3-19 School,30,Welsh establishment,1,Open,2023-09-01,NaT,...,<NA>,<NA>,999,<NA>,<NA>,,999999999,999999999,679-5500,<NA>
402469,681,Cardiff,2333,Ysgol Gynradd Groes-Wen Primary,30,Welsh establishment,1,Open,2023-09-01,NaT,...,(England/Wales) Rural village,<NA>,999,<NA>,<NA>,,W02000380,W01001729,681-2333,<NA>
402470,668,Pembrokeshire,2398,Ysgol Bro Penfro,30,Welsh establishment,4,Proposed to open,2024-09-01,NaT,...,(England/Wales) Rural town and fringe,<NA>,999,<NA>,<NA>,,W02000140,W01000607,668-2398,<NA>


Merge required GIAS, census, sen, cdc, PFI, and arr data with the base academy data

In [47]:
academies = pre_processing.build_academy_data('data/academy_master_list.csv', 
                                              current_year, schools, census, sen, cdc, 
                                              academy_ar, ks2, ks4)

In [48]:
#academies.to_csv('output/pre-processing/academies.csv', columns=output_schemas.academies_output)
academies.sort_index()

,URN,Company Registration Number,Incorporation Date,Academy Trust UPIN,Trust UKPRN,Trust Name,Academy Name,Academy UPIN,Trust Type,Date Opened,...,Other costs_Staff development and training_Per Unit,Other costs_Staff-related insurance_Per Unit,Other costs_Supply teacher insurance_Per Unit,Other costs_Rent and rates_Per Unit,Other costs_Special facilities_Per Unit,Other costs_Other insurance premiums_Per Unit,Other costs_Interest charges for loan and bank_Per Unit,Other costs_Direct revenue financing_Per Unit,Other costs_PFI charges_Per Unit,Other costs_Total_Per Unit
0,148853,10817580,2017-06-14 00:00:00.0000000,139821,10064612,1Excellence Multi Academy Trust,Evenwood Church of England Primary School,163480,Multi Academy Trust (MAT),2021-12-01 00:00:00.0000000,...,88.888889,0.0,0.0,11.111111,0.0,0.0,0.0,0.0,0.0,277.777778
1,144542,10817580,2017-06-14 00:00:00.0000000,139821,10064612,1Excellence Multi Academy Trust,Pentland Primary School,138448,Multi Academy Trust (MAT),2017-07-01 00:00:00.0000000,...,59.016393,0.0,0.0,72.131148,0.0,0.0,0.0,0.0,0.0,190.163934
2,144551,10817580,2017-06-14 00:00:00.0000000,139821,10064612,1Excellence Multi Academy Trust,St Mark's Church of England Primary School - S...,138465,Multi Academy Trust (MAT),2017-07-01 00:00:00.0000000,...,56.561086,0.0,0.0,20.361991,0.0,0.0,0.0,0.0,0.0,133.484163
3,148854,10817580,2017-06-14 00:00:00.0000000,139821,10064612,1Excellence Multi Academy Trust,St Michael's Church of England Primary School ...,163504,Multi Academy Trust (MAT),2021-12-01 00:00:00.0000000,...,70.866142,0.0,0.0,15.748031,0.0,0.0,0.0,0.0,0.0,283.464567
4,136730,07595434,2011-04-07 00:00:00.0000000,134890,10058682,5 Dimensions Trust,Shenley Brook End School,119734,Multi Academy Trust (MAT),2011-05-01 00:00:00.0000000,...,21.890016,0.0,0.0,41.110518,0.0,27.762947,0.0,0.0,0.0,117.458623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10139,147031,07559293,2011-03-10 00:00:00.0000000,136062,10058621,Zenith Multi Academy Trust,Castle View School - Canvey Island,151873,Multi Academy Trust (MAT),2014-10-01 00:00:00.0000000,...,32.223416,0.0,0.0,50.483351,0.0,21.482277,0.0,0.0,1257.787325,1380.236305
10140,145812,07559293,2011-03-10 00:00:00.0000000,136062,10058621,Zenith Multi Academy Trust,Laindon Park Primary School & Nursery,140554,Multi Academy Trust (MAT),2018-09-01 00:00:00.0000000,...,29.850746,0.0,0.0,19.900498,0.0,19.900498,0.0,0.0,0.0,74.626866
10141,138865,07559293,2011-03-10 00:00:00.0000000,136062,10058621,Zenith Multi Academy Trust,The James Hornsby School,121878,Multi Academy Trust (MAT),2012-10-01 00:00:00.0000000,...,46.610169,0.0,0.0,28.601695,0.0,25.423729,0.0,0.0,0.0,116.525424
10142,136577,07559293,2011-03-10 00:00:00.0000000,136062,10058621,Zenith Multi Academy Trust,The King John School,119652,Multi Academy Trust (MAT),2011-04-01 00:00:00.0000000,...,12.664394,0.0,0.0,27.277155,0.0,25.328787,0.0,0.0,0.0,83.779834


Merge required census and cdc data to the maintained schools data set

In [49]:
# Load raw list from CSV
maintained_schools = pre_processing.build_maintained_school_data('data/maintained_schools_master_list.csv',current_year, schools, census, sen, cdc, ks2, ks4)

In [50]:
maintained_schools.to_csv('output/pre-processing/maintained_schools.csv', columns=output_schemas.maintained_schools_output)
#maintained_schools

## Federation Capture




In [51]:
(hard_federations, soft_federations) = pre_processing.build_federations_data('data/gias_all_links.csv', maintained_schools)

In [52]:
hard_federations.to_csv('output/pre-processing/hard_federations.csv')
hard_federations

,LAEstab,FederationName,FederationUid,Closed Date,Members
URN,,,,,
100234,2042450,"The Viridis Federation of Orchard, Southwold &...",1652,<NA>,"[100234, 100242, 131141]"
100250,2042779,The LEAP Federation,17386,<NA>,"[100250, 100261, 130303]"
100258,2042864,New Wave Federation,1589,<NA>,"[100241, 100248, 100258]"
100263,2043358,Primary Advantage Federation,1473,<NA>,"[100224, 100225, 100232, 100263, 100266, 10026..."
101251,3021000,Barnet Early Years Alliance,5603,<NA>,"[101251, 101252, 101254]"
...,...,...,...,...,...
125246,9363928,Newlands CofE School Federation,17520,<NA>,"[125199, 125246]"
125288,9365206,The Federation of Holy Trinity and Pewley Down...,1025,<NA>,"[125288, 136755]"
125473,9367053,Federation of Manor Mead and Walton Leigh Schools,15744,<NA>,"[125468, 125473]"


In [53]:
soft_federations.to_csv('output/pre-processing/soft_federations.csv')
soft_federations[['LAEstab']]

,LAEstab
URN,
100391,2061104
100472,2071010
105861,3547006
108960,8012073
109417,8221009
109864,8692131
109964,8693026
111246,8965204
113172,8782249


In [54]:
# amalgamate school data

list_of_laestabs = maintained_schools["LAEstab"][maintained_schools["Lead school in federation"]!="0"]
list_of_ukprns = maintained_schools["UKPRN"][maintained_schools["Lead school in federation"]!="0"]
mappings = dict(zip(list_of_laestabs,list_of_ukprns))

maintained_schools["Federation Lead School UKPRN"] = maintained_schools["Lead school in federation"].map(mappings)
maintained_schools = pd.merge(maintained_schools, hard_federations[['FederationName']], how='left', on='URN')
maintained_schools.rename(columns = {"FederationName":"Federation Name"}, inplace = True)
all_school_data = pd.concat((maintained_schools, academies))

all_school_data.to_csv('output/pre-processing/schools.csv', columns=output_schemas.schools_table_output)



In [55]:
maintained_schools[["UKPRN","Federation Lead School UKPRN","LAEstab","Lead school in federation"]][maintained_schools['Lead school in federation']!="0"]


,UKPRN,Federation Lead School UKPRN,LAEstab,Lead school in federation
URN,,,,
101997,10046090,10046090,3082040,3082040
109923,10079844,10079844,8692231,8692231
109932,10079843,10079843,8692240,8692240
113223,10076081,10076081,8782446,8782446
111246,10075913,10075913,8965204,8965204
...,...,...,...,...
121602,10078107,10078107,8153289,8153289
121608,10072014,10072014,8153306,8153306
108960,10069766,10069766,8012073,8012073


### Timing Keep at the bottom

In [56]:
print(f'Processing Time: {time.time() - start_time} seconds')

Processing Time: 40.77061891555786 seconds
